<a href="https://colab.research.google.com/github/PyBlin/Study/blob/main/PyML/Chapter2_ScikitLearn/Chap2_4_ModelSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4.Model Selection 모듈 소개

## 4.1 train_test_split()

In [1]:
# 테스트 데이터셋을 이용하지 않고 무엇이 문제인지 확인해보기
# 한마디로 학습과 예측을 동일한 데이터셋으로 수행
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris = load_iris()
dt_clf = DecisionTreeClassifier()
train_data = iris.data
train_label = iris.target
dt_clf.fit(train_data, train_label)

# 학습 데이터셋으로 예측 수행
pred = dt_clf.predict(train_data)
print(f"예측 정확도 : {accuracy_score(train_label, pred)}")

예측 정확도 : 1.0


* 정확도가 100%...? 이건 그냥 컨닝한거나 다름없습니다.
* 이제 `train_test_split()`으로 학습용과 테스트용 데이터셋을 분리해봅시다.

In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

iris_data = load_iris()
dt_clf = DecisionTreeClassifier()

X_train, X_test, y_train, y_test = train_test_split(iris_data.data, 
                                                    iris_data.target, 
                                                    test_size=0.3, 
                                                    random_state=121)

* `train_test_split(피처 데이터셋, 레이블 데이터셋)`
* `test_size` : 테스트 데이터셋 크기를 결정합니다. (default=0.25)
* `shuffle` : 데이터 분리 전에 데이터를 섞을지 결정합니다. (default=True)
* `random_state` : 지정하지 않으면 수행할 때마다 다른 학습용/테스트용 데이터셋을 생성합니다.

In [3]:
# 의사결정나무를 학습하고 예측 정확도를 측정
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print(f"예측 정확도 : {accuracy_score(y_test, pred):.4f}")

예측 정확도 : 0.9556


## 4.2 교차 검증 (Cross Validation)

### 4.2.1 K-fold 교차 검증

In [4]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris_data = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier()

# 5개의 폴드셋으로 분리하는 KFold 객체와 폴드셋별 정확도를 담을 리스트 객체 생성
kfold = KFold(n_splits=5)
cv_accuracy = []
print(f"붓꽃 데이터셋 크기 : {features.shape[0]}")

붓꽃 데이터셋 크기 : 150


* 전체 데이터셋 150을 5등분 하면
    - 학습용 데이터셋 : 120개
    - 검증용 데이터셋 : 30개

In [5]:
n_iter = 0

# split()를 호출하면 폴드별 학습용/검증용 테스트의 row index를 array로 반환
for train_index, test_index in kfold.split(features):

    # kfold.split()으로 반환된 인덱스를 이용해 학습용/검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]

    # 학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    n_iter += 1

    # 반복할 때마다 정확도 측정
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print(f"\n#{n_iter} 교차 검증 정확도 : {accuracy}, \
    학습 데이터 크기 : {train_size}, \
    검증 데이터 크기 : {test_size}")
    print(f"#{n_iter} 검증셋 인덱스 : {test_index}")
    cv_accuracy.append(accuracy)

# 개별 iteration별 정확도를 합하여 평균 정확도 계산
print(f"\n## 평균 검증 정확도 : {np.mean(cv_accuracy)*100:.2f} %")


#1 교차 검증 정확도 : 1.0,     학습 데이터 크기 : 120,     검증 데이터 크기 : 30
#1 검증셋 인덱스 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 : 1.0,     학습 데이터 크기 : 120,     검증 데이터 크기 : 30
#2 검증셋 인덱스 : [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 : 0.8333,     학습 데이터 크기 : 120,     검증 데이터 크기 : 30
#3 검증셋 인덱스 : [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 : 0.9333,     학습 데이터 크기 : 120,     검증 데이터 크기 : 30
#4 검증셋 인덱스 : [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 : 0.8333,     학습 데이터 크기 : 120,     검증 데이터 크기 : 30
#5 검증셋 인덱스 : [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도 : 92.00 %


### 4.2.2. Stratified K-fold

* 불균형한 분포도를 가진 레이블 데이터 집합을 위한 K-fold 방식
* ex) 대출사기 : 0.0001% 확률로 대출 사기 레이블이 존재

In [6]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
iris_df = pd.DataFrame(data=iris_data, columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

* 레이블 값이 0(Setosa), 1(Versicolor), 2(Virginica) 모두 50개로 동일합니다.
* 먼저 KFold를 사용하여 문제점을 알아봅시다.

In [7]:
# 3개의 폴드셋을 KFold로 생성
# 교차 검증을 할 때마다 생성되는 학습용/검증용 레이블 데이터 값의 분포도 확인
from sklearn.model_selection import KFold

kfold = KFold(n_splits=3)
n_iter = 0

for train_index, test_index in kfold.split(iris_df):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print(f"## 교차 검증 : {n_iter}")
    print(f"학습 레이블 데이터 분포 : \n{label_train.value_counts()}")
    print(f"검증 레이블 데이터 분포 : \n{label_test.value_counts()}\n")

## 교차 검증 : 1
학습 레이블 데이터 분포 : 
2    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
0    50
Name: label, dtype: int64

## 교차 검증 : 2
학습 레이블 데이터 분포 : 
2    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
1    50
Name: label, dtype: int64

## 교차 검증 : 3
학습 레이블 데이터 분포 : 
1    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
2    50
Name: label, dtype: int64



* 학습 레이블과 검증 레이블이 완전히 다른 값으로 추출되었습니다.
    - 학습한 데이터를 바탕으로 검증해야하는데, 생뚱맞은 값이 나왔습니다.
    - 이렇게 하면 검증 예측 정확도는 0% 입니다 ㅠㅠ

* Stratified KFold는 이런 문제를 해결해줍니다.
* 이번에는 Stratified KFold를 사용해봅시다.
* 사용 방법은 KFold와 거의 비슷하지만, split() 인자에 레이블 데이터셋을 추가해야 합니다.

In [8]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter = 0

for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print(f"## 교차 검증 : {n_iter}")
    print(f"학습 레이블 데이터 분포 : \n{label_train.value_counts()}")
    print(f"검증 레이블 데이터 분포 : \n{label_test.value_counts()}\n")

## 교차 검증 : 1
학습 레이블 데이터 분포 : 
2    34
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
1    17
0    17
2    16
Name: label, dtype: int64

## 교차 검증 : 2
학습 레이블 데이터 분포 : 
1    34
2    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
2    17
0    17
1    16
Name: label, dtype: int64

## 교차 검증 : 3
학습 레이블 데이터 분포 : 
0    34
2    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
2    17
1    17
0    16
Name: label, dtype: int64



* 학습 레이블과 검증 레이블 데이터 값의 분포도가 동일하게 할당되었습니다.
* 이제 StratifiedKFold를 이용해 붓꽃 데이터를 교차 검증해 봅시다.

In [9]:
df_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits=3)
n_iter = 0
cv_accuracy = []

# split() 호출 시 반드시 레이블 데이터셋 추가 입력
for train_index, test_index in skfold.split(features, label):
    
    # split()으로 반환된 인덱스를 이용해 학습용/검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]

    # 학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)

    # 반복할 때마다 정확도 측정
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print(f"\n#{n_iter} 교차 검증 정확도 : {accuracy}, \
    학습 데이터 크기 : {train_size}, \
    검증 데이터 크기 : {test_size}")
    print(f"#{n_iter} 검증셋 인덱스 : {test_index}")
    cv_accuracy.append(accuracy)

# 교차 검증별 정확도 및 평균 정확도 계산
print(f"\n## 교차 검증별 정확도 : {np.round(cv_accuracy, 4)}")
print(f"\n## 평균 검증 정확도 : {np.mean(cv_accuracy):.4f}")


#1 교차 검증 정확도 : 0.98,     학습 데이터 크기 : 100,     검증 데이터 크기 : 50
#1 검증셋 인덱스 : [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

#2 교차 검증 정확도 : 0.92,     학습 데이터 크기 : 100,     검증 데이터 크기 : 50
#2 검증셋 인덱스 : [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

#3 교차 검증 정확도 : 0.98,     학습 데이터 크기 : 100,     검증 데이터 크기 : 50
#3 검증셋 인덱스 : [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도 : [0.98 0.92 0.98]

## 평균 검증 정확도 : 0.9600


* 일반적으로 분류에서의 교차 검증은 Stratified K-fold로 분할되어야 합니다.
* 회귀에서는 결정값이 연속된 숫자값이기 때문에 결정값별로 분포를 정하는 의미가 없습니다.

### 4.2.3 cross_val_score()

* 주요 파라미터
    * `estimator` : Classifier or Regressor
    * `X` : feature data set
    * `y=None` : label data set
    * `scoring=None` : 예측 성능 평가 지표
    * `cv=None` : 교차 검증 폴드 수

* 기타 파라미터
    * `n_jobs=1`
    * `verbose=0`
    * `fit_params=None`
    * `pre_dispatch='2*n_jobs'`

In [10]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, cross_validate
import numpy as np

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도(accuracy), 교차 검증셋은 3개
scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3)
print(f"교차 검증별 정확도 : {np.round(scores, 4)}")
print(f"평균 검증 정확도 : {np.round(np.mean(scores), 4)}")

교차 검증별 정확도 : [0.98 0.94 0.98]
평균 검증 정확도 : 0.9667


* `cross_val_score()` API는 내부에서 estimator를 학습(fit), 예측(predict), 평가(evaluation)시켜주므로 간단하게 교차 검증을 수행할 수 있습니다.
* `cross_val_score()`는 내부적으로 Stratified K-fold를 이용합니다.
* 비슷한 API로 `cross_validate()`가 있습니다.

## 4.3 Grid Search Cross Validation

* 교차 검증과 최적 하이퍼 파라미터 튜닝을 한 번에!!
* Grid는 "격자"라는 뜻으로, 파라미터를 촘촘하게 입력하면서 테스트하는 방식입니다.

In [11]:
# decision tree algorithm's optimal hyper parameter
grid_parameters = {'max_depth':[1, 2, 3], 'min_samples_split':[2, 3]}
grid_parameters

{'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]}

* depth 1 --> 2, 3
* depth 2 --> 2, 3
* depth 3 --> 2, 3
    - 총 6회에 걸쳐 파라미터를 순차적으로 바꿔 실행하면서 최적의 파라미터와 수행 결과를 도출합니다.
    - 시간이 오래 걸립니다.
    - cv=3 이라면 총 3 x 6 = 18회의 학습/평가가 수행됩니다.

In [12]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

iris_data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, 
                                                    iris_data.target, 
                                                    test_size=0.2, 
                                                    random_state=121)
dtree = DecisionTreeClassifier()

# 파라미터를 딕셔너리 형태로 설정
parameters = {'max_depth':[1, 2, 3], 'min_samples_split':[2, 3]}

* GridSearchCV 주요 생성자
    * `estimator` : Classifier, Regressor, Pipeline
    * `param_grid` : "key + 리스트값"을 가지는 딕셔너리가 주어집니다.
        - estimator 튜닝을 위해 파라미터명과 사용될 여러 파라미터값을 지정합니다.
    * `scoring` : 보통 문자열('accuracy')로 지정하나 별도의 함수도 지정할 수 있습니다.
    * `cv`
    * `refit=True` : True로 생성 시 가장 최적의 하이퍼 파라미터를 찾은 후 입력된 estimator 객체를 해당 하이퍼 파라미터로 재학습시킵니다.

In [13]:
import pandas as pd

# param_grid의 하이퍼 파라미터를 3개의 train, test set fold로 나누어 테스트 수행
# refit=True(default) : 가장 좋은 파라미터 설정으로 재학습
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True)

# 붓꽃 학습 데이터로 param_grid 하이퍼 파라미터를 순차적으로 학습/평가
grid_dtree.fit(X_train, y_train)
grid_dtree.cv_results_

{'mean_fit_time': array([0.00143758, 0.00091497, 0.00072885, 0.00052786, 0.00051824,
        0.00050831]),
 'mean_score_time': array([0.00075587, 0.00057991, 0.00037225, 0.00032401, 0.00035961,
        0.00035429]),
 'mean_test_score': array([0.7       , 0.7       , 0.95833333, 0.95833333, 0.975     ,
        0.975     ]),
 'param_max_depth': masked_array(data=[1, 1, 2, 2, 3, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[2, 3, 2, 3, 2, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 1, 'min_samples_split': 2},
  {'max_depth': 1, 'min_samples_split': 3},
  {'max_depth': 2, 'min_samples_split': 2},
  {'max_depth': 2, 'min_samples_split': 3},
  {'max_depth': 3, 'min_samples_split': 2},
  {'max_depth': 3, 'min_samples_split': 3}],
 'rank_test_score': array([5, 5, 3, 3, 1, 1], 

* `cv_result_` : `fit()` 수행 후 학습 데이터를 폴딩셋으로 분할하여 `param_grid`에 기술된 하이퍼 
파라미터를 순차적으로 변경하면서 학습/평가를 수행하고 그 결과를 `cv_result_`
속성에 기록합니다.

In [14]:
# GridSearchCV 결과를 추출해 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 
           'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


* params : 수행할 때마다 적용된 하이퍼 파라미터값입니다.
* mean_test_score : 개별 하이퍼 파라미터별로 폴딩 테스트셋에 대해 총 수행한 평가 평균값입니다.
* rank_test_score : 하이퍼 파라미터별로 성능이 좋은 순위입니다. 1일 때 가장 최적의 하이퍼 파라미터입니다.
* split[i]_test_score : cv 값으로 생성된 각각의 폴딩셋(cv=3 --> 3개의 폴딩셋)입니다.

In [15]:
print(f"Grid Search CV 최적 파라미터 : {grid_dtree.best_params_}")
print(f"Grid Search CV 최고 정확도 : {grid_dtree.best_score_:.4f}")

Grid Search CV 최적 파라미터 : {'max_depth': 3, 'min_samples_split': 2}
Grid Search CV 최고 정확도 : 0.9750


* `best_params_` : `fit()` 수행 후 최고 성능의 하이퍼 파라미터값을 `best_params_` 속성에 기록합니다.
* `best_score_` : `fit()` 수행 후 최고 성능의 하이퍼 파라미터값의 평가 결과값을 `best_score_` 속성에 기록합니다.
    - (즉, `cv_result_`의 `rank_test_score=1`일 때의 값입니다.)

In [16]:
# Grid Search CV의 refit으로 이미 학습된 estimator 반환
estimator = grid_dtree.best_estimator_

# Grid Search CV의 best_estimator_는 이미 최적 학습이 되어 별도 학습이 필요 없음
pred = estimator.predict(X_test)
print(f"Test data set's accuracy : {accuracy_score(y_test, pred):.4f}")

Test data set's accuracy : 0.9667


* `best_estimator_` : `refit=True`이면 GridSearchCV가 최적 성능의 하이퍼 파라미터로 Estimator를 학습하여 `best_esimator_`로 저장합니다.